In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('f1Practice').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/16 09:58:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

constructor_schema = StructType(fields=[
    StructField('constructorId', IntegerType(), True),
    StructField('constructorRef', StringType(), True),
    StructField('name', StringType(), True),
    StructField('nationality', StringType(), True),
    StructField('url', StringType(), True)
])

In [10]:
from pathlib import Path

constructor_bronze_path = Path.cwd() / 'bronze' / 'constructors.json' 
df = spark.read.schema(constructor_schema).json(str(constructor_bronze_path))

In [12]:
from pyspark.sql.functions import col

selected_df = df.select(
    col('constructorId').alias('constructor_id'),
    col('constructorRef').alias('constructor_ref'),
    col('name'),
    col('nationality')
)

In [16]:
from pyspark.sql.functions import current_timestamp

final_df = selected_df.withColumn('ingestion_date', current_timestamp())

constructor_silver_path = Path.cwd() / 'silver' / 'constructors'

final_df.write.mode('overwrite').parquet(str(constructor_silver_path))